# Reto Viva Aerobus
## Equipo: New Rochelle Challengers
## Mesa: 52
### Diego Esparza Hurtado
### Alejandro Lizarraga Vizcarra
### Diego Linarez
### Ricardo Vladimir

## Limpieza:

### Librerias:

In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

predecir numero de pasajeros primero
salida, llegada, estacionalidad (fecha) dia/tarde/noche (horario) Obtener basado en horarios


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression

Modificaciones para observar las tablas:

In [88]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 100

### Lectura de archivos:

In [89]:
dataFlights = pd.read_csv('Flights.csv') # Datos de vuelos.
dataSales = pd.read_csv('Sales.csv') # Datos de ventas de productos.

In [90]:
dataFlights.head() # Primeros cinco registros de vuelos.

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0


In [91]:
dataSales.head() # Primeros cinco registros de ventas de productos.

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0


### Información básica de los datos:
Se buscan observar la cantidad de datos total, total de datos no nulos y el tipo de dato de cada columna.

In [92]:
dataFlights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245753 entries, 0 to 245752
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Flight_ID         245753 non-null  object 
 1   Aeronave          165363 non-null  object 
 2   DepartureStation  245752 non-null  object 
 3   ArrivalStation    245752 non-null  object 
 4   Destination_Type  245752 non-null  object 
 5   Origin_Type       245752 non-null  object 
 6   STD               245753 non-null  object 
 7   STA               245753 non-null  object 
 8   Capacity          245753 non-null  int64  
 9   Passengers        122228 non-null  float64
 10  Bookings          122228 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 20.6+ MB


In [93]:
dataSales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564361 entries, 0 to 2564360
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Flight_ID    object 
 1   ProductType  object 
 2   ProductName  object 
 3   Quantity     int64  
 4   TotalSales   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 97.8+ MB


### Eliminar registros repetidos de 'Flight_ID'

In [94]:
dataFlights['Flight_ID'].unique().size/dataFlights['Flight_ID'].size # Proporción de registros con 'Flight_ID' repetido.

0.9686758656048959

In [95]:
dataFlights = dataFlights.drop_duplicates(subset=['Flight_ID']) # Se eliminan los duplicados.

### Modificaciones a las fechas de salida y llegada de vuelos:

In [96]:
dataFlights['STD'] = pd.to_datetime(dataFlights['STD']) # Cambio de formato a datetime para 'STD'.
dataFlights['STA'] = pd.to_datetime(dataFlights['STA']) # Cambio de formato a datetime para 'STA'.

In [97]:
# Se extraen los minutos, hora, día de la semana, día del mes, mes y año de 'STD'.
dataFlights['STD_year'] = dataFlights['STD'].dt.year
dataFlights['STD_month'] = dataFlights['STD'].dt.month
dataFlights['STD_day'] = dataFlights['STD'].dt.day
dataFlights['STD_day_week'] = dataFlights['STD'].dt.day_name()
dataFlights['STD_hour'] = dataFlights['STD'].dt.hour
dataFlights['STD_minute'] = dataFlights['STD'].dt.minute

# Se extraen los minutos, hora, día de la semana, día del mes, mes y año de 'STA'.
dataFlights['STA_year'] = dataFlights['STA'].dt.year
dataFlights['STA_month'] = dataFlights['STA'].dt.month
dataFlights['STA_day'] = dataFlights['STA'].dt.day
dataFlights['STA_day_week'] = dataFlights['STA'].dt.day_name()
dataFlights['STA_hour'] = dataFlights['STA'].dt.hour
dataFlights['STA_minute'] = dataFlights['STA'].dt.minute

# Se calcula la duración de tiempo en segundos de cada vuelo. 
dataFlights['Flight_Duration'] = dataFlights['STA'] - dataFlights['STD']
dataFlights['Flight_Duration'] = dataFlights['Flight_Duration'].dt.total_seconds().astype('int64')

In [98]:
dataFlights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,2023,10,19,Thursday,11,40,2023,10,19,Thursday,14,25,9900
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,2023,7,3,Monday,0,55,2023,7,3,Monday,4,55,14400
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2024,2,16,Friday,17,10,2024,2,16,Friday,17,55,2700
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2023,6,26,Monday,15,15,2023,6,26,Monday,15,55,2400
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,2023,2,10,Friday,8,40,2023,2,10,Friday,9,50,4200


In [99]:
dataFlights.describe()

,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_hour,STA_minute,Flight_Duration
count,238055,238055,238055.000000,119645.000000,119645.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000,238055.000000
mean,2023-12-30 04:44:28.233810944,2023-12-30 06:48:16.472748544,204.339795,183.818680,114.062552,2023.504467,6.399618,15.681511,13.195505,25.096553,2023.504505,6.401008,15.683603,13.808910,28.238672,7428.238936
min,2023-01-02 00:20:00,2023-01-02 02:45:00,178.000000,1.000000,1.000000,2023.000000,1.000000,1.000000,0.000000,0.000000,2023.000000,1.000000,1.000000,0.000000,0.000000,900.000000
25%,2023-07-13 12:45:00,2023-07-13 14:50:00,186.000000,162.000000,93.000000,2023.000000,4.000000,8.000000,8.000000,10.000000,2023.000000,4.000000,8.000000,9.000000,15.000000,5100.000000
50%,2023-12-30 06:20:00,2023-12-30 08:15:00,186.000000,185.000000,113.000000,2023.000000,6.000000,16.000000,13.000000,25.000000,2023.000000,6.000000,16.000000,14.000000,30.000000,6000.000000
75%,2024-06-19 08:15:00,2024-06-19 09:57:30,240.000000,211.000000,134.000000,2024.000000,9.000000,23.000000,18.000000,40.000000,2024.000000,9.000000,23.000000,19.000000,45.000000,9300.000000
max,2025-02-28 06:05:00,2025-02-28 07:35:00,240.000000,271.000000,219.000000,2025.000000,12.000000,31.000000,23.000000,59.000000,2025.000000,12.000000,31.000000,23.000000,59.000000,98400.000000
std,NaN,NaN,25.585660,37.627153,30.358190,0.513910,3.313200,8.812224,5.585557,17.577292,0.513910,3.313040,8.811498,6.028976,17.165927,3658.162223


### Se separan en dos conjuntos (2023 y 2024) los datos para tener los datos para entrenamiento del modelo y los datos a utilizar en las predicciones.

In [100]:
dataFlights2023 = dataFlights[dataFlights['STD'] < '2024-01-01'] # Datos para entrenamiento.
dataFlights2024 = dataFlights[(dataFlights['STD'] >= '2024-01-01') & (dataFlights['STD'] <= '2024-05-05')] # Datos para predicción.

#### Se revisan valores Na en los datos para entrenamiento.

In [101]:
dataFlights2023.isna().sum()

Flight_ID             0
Aeronave            492
DepartureStation      1
ArrivalStation        1
Destination_Type      1
Origin_Type           1
STD                   0
STA                   0
Capacity              0
Passengers            0
Bookings              0
STD_year              0
STD_month             0
STD_day               0
STD_day_week          0
STD_hour              0
STD_minute            0
STA_year              0
STA_month             0
STA_day               0
STA_day_week          0
STA_hour              0
STA_minute            0
Flight_Duration       0
dtype: int64

In [102]:
dataFlights2023.isna().sum()/dataFlights2023.shape[0] # Proporción de los datos Na en cada columna.

Flight_ID           0.000000
Aeronave            0.004112
DepartureStation    0.000008
ArrivalStation      0.000008
Destination_Type    0.000008
Origin_Type         0.000008
STD                 0.000000
STA                 0.000000
Capacity            0.000000
Passengers          0.000000
Bookings            0.000000
STD_year            0.000000
STD_month           0.000000
STD_day             0.000000
STD_day_week        0.000000
STD_hour            0.000000
STD_minute          0.000000
STA_year            0.000000
STA_month           0.000000
STA_day             0.000000
STA_day_week        0.000000
STA_hour            0.000000
STA_minute          0.000000
Flight_Duration     0.000000
dtype: float64

In [103]:
dataFlights2023 = dataFlights2023.dropna() # Se eliminan los datos Na.

In [104]:
dataFlights2023.isna().sum()

Flight_ID           0
Aeronave            0
DepartureStation    0
ArrivalStation      0
Destination_Type    0
Origin_Type         0
STD                 0
STA                 0
Capacity            0
Passengers          0
Bookings            0
STD_year            0
STD_month           0
STD_day             0
STD_day_week        0
STD_hour            0
STD_minute          0
STA_year            0
STA_month           0
STA_day             0
STA_day_week        0
STA_hour            0
STA_minute          0
Flight_Duration     0
dtype: int64

Posibles variables:
- DepartureStation
- ArrivalStation
- Destination_Type?
- Origin_Type?
- STD
- STA
- Capacity?
- Passengers
- Flight_Duration

### Transformación de variables cualitativas a cuantitativas.

In [105]:
from sklearn.preprocessing import LabelEncoder

In [106]:
dataFlights2023_encoded = dataFlights2023.copy()

label_encoders = {} # Inicialización del enconder.

# Se genera el label encoder en cada columna.
for col in ['Aeronave', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'STD_day_week', 'STA_day_week']:
    label_encoders[col] = LabelEncoder()
    dataFlights2023_encoded[col] = label_encoders[col].fit_transform(dataFlights2023_encoded[col])

dataFlights2023_encoded # Datos modificados

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration
0,ab954014077430bd842cfa305a55c0f8,39,15,21,0,1,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,2023,10,19,4,11,40,2023,10,19,4,14,25,9900
1,efd86c996035dacdca7a0ccb2560dda1,61,34,17,3,0,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,2023,7,3,1,0,55,2023,7,3,1,4,55,14400
3,dd0fad3248951d2f71d63e6279aeaa4b,37,18,14,3,1,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2023,6,26,1,15,15,2023,6,26,1,15,55,2400
4,d0987ee648eea254063bfe2b39571b67,13,22,0,4,1,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,2023,2,10,0,8,40,2023,2,10,0,9,50,4200
5,3b5df8805161ea827d2f2e4298c38e06,39,6,13,4,3,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,2023,9,7,4,17,10,2023,9,7,4,18,5,3300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245689,9bbd9f3a2bbcaeae864d17e3ee544d2c,6,15,40,4,1,2023-12-06 12:50:00,2023-12-06 16:10:00,186,191.0,119.0,2023,12,6,6,12,50,2023,12,6,6,16,10,12000
245718,d9ef769565064b1ff1ad90d6d18f69cb,83,10,40,4,1,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0,2023,12,26,5,6,20,2023,12,26,5,10,10,13800
245745,598035669a4f0fa2ce1207977932df9e,7,18,40,4,1,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0,2023,12,28,4,10,25,2023,12,28,4,13,50,12300
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,27,34,40,4,0,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0,2023,12,29,0,7,30,2023,12,29,0,14,50,26400


## Modelos para predicción de volumen de pasajeros:

Se dividen las variables a utilizar como X (Independientes) y variable a obtener Y (Dependiente).

X:
- 'DepartureStation'
- 'ArrivalStation'
- 'Destination_Type'
- 'Origin_Type'
- 'Capacity'
- 'STD_month'
- 'STD_day'
- 'STD_day_week'
- 'STD_hour'
- 'Flight_Duration'

Y:
- 'Passengers'

In [107]:
X = dataFlights2023_encoded[['DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'Capacity', 'STD_month', 'STD_day', 'STD_day_week', 'STD_hour', 'Flight_Duration']]
y = dataFlights2023_encoded['Passengers']

Se generan los grupos para el entrenamiento y comprobación de los modelos y se estandarizan:

In [108]:
from sklearn.model_selection import train_test_split # Division de grupos para entrenamiento y pruebas.
from sklearn.preprocessing import StandardScaler # Estandarización
from sklearn.metrics import mean_squared_error, r2_score # Corroboración del modelo.

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [110]:
# Estandarización de datos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Random Forest Regressor

In [111]:
from sklearn.ensemble import RandomForestRegressor

In [112]:
# Se elige el modelo y se entrena.
modelFlights = RandomForestRegressor()
modelFlights.fit(X_train_scaled, y_train)

# Se generan las predicciones.
predictions = modelFlights.predict(X_test_scaled)

# Se calculan los errores:
mse = mean_squared_error(y_test, predictions)
print("Error medio cuadrado:", mse)
rmse = np.sqrt(mse)
print("Error medio:", rmse)

# R^2
r2 = r2_score(y_test, predictions)

# R^2
print("R^2:", r2)

Error medio cuadrado: 412.58380935600866
Error medio: 20.312159150518898
R^2: 0.7048777207818265


### BaggingRegressor

In [113]:
from sklearn.ensemble import BaggingRegressor

In [114]:
# Se elige el modelo y se entrena.
model = BaggingRegressor()
model.fit(X_train_scaled, y_train)

# Se generan las predicciones.
predictions = model.predict(X_test_scaled)

# Se calculan los errores:
mse = mean_squared_error(y_test, predictions)
print("Error medio cuadrado:", mse)
rmse = np.sqrt(mse)
print("Error medio:", rmse)

# R^2
r2 = r2_score(y_test, predictions)

# R^2
print("R^2:", r2)

Error medio cuadrado: 457.5197159170024
Error medio: 21.389710515035084
R^2: 0.6727349491502518


### Predicciones 2024:

In [115]:
dataFlights2024_encoded = dataFlights2024.copy()

for col in ['DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'STD_day_week', 'STA_day_week']:
    dataFlights2024_encoded[col] = label_encoders[col].transform(dataFlights2024_encoded[col])

In [116]:
X_pred = dataFlights2024_encoded[['DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'Capacity', 'STD_month', 'STD_day', 'STD_day_week', 'STD_hour', 'Flight_Duration']]

In [117]:
# Estandarización de datos.
scaler = StandardScaler()
X_finalpred = scaler.fit_transform(X_pred)

In [118]:
# Se generan las predicciones.
y_predictions = modelFlights.predict(X_finalpred)

In [166]:
dataFlights2024_encoded['Passengers'] = y_predictions

In [167]:
dataFlights2024_encoded

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration,Expected Passengers
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,18,14,3,1,2024-02-16 17:10:00,2024-02-16 17:55:00,220,193.68,NaN,2024,2,16,0,17,10,2024,2,16,0,17,55,2700,193.68
19,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,22,0,4,1,2024-01-12 08:20:00,2024-01-12 09:20:00,186,178.40,NaN,2024,1,12,0,8,20,2024,1,12,0,9,20,3600,178.40
25,85d3b182431d323b856e50e249689981,XA-VIY,15,5,4,1,2024-03-18 10:30:00,2024-03-18 13:25:00,186,168.98,NaN,2024,3,18,1,10,30,2024,3,18,1,13,25,10500,168.98
26,04e83b220f5513bd6cc636784dcddec6,XA-VXC,34,17,3,0,2024-04-05 00:50:00,2024-04-05 04:45:00,240,196.28,NaN,2024,4,5,0,0,50,2024,4,5,0,4,45,14100,196.28
28,0dbc47cfbd0af71db43ef3aa60ca1ced,XA-VII,22,21,0,1,2024-02-24 07:35:00,2024-02-24 09:20:00,186,182.82,NaN,2024,2,24,2,7,35,2024,2,24,2,9,20,6300,182.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245731,1d178f4a133f821d3371d21870a1b138,XA-VIS,10,40,4,1,2024-01-13 06:20:00,2024-01-13 10:10:00,186,166.69,NaN,2024,1,13,2,6,20,2024,1,13,2,10,10,13800,166.69
245733,5a3536b189cf617f594037d9dbc42797,XA-VYE,18,40,4,1,2024-03-21 09:55:00,2024-03-21 13:10:00,180,141.63,NaN,2024,3,21,4,9,55,2024,3,21,4,13,10,11700,141.63
245737,184e0731624c96b02fd250d4dd9a6b52,XA-VXD,15,40,4,1,2024-01-27 12:50:00,2024-01-27 15:55:00,240,183.56,NaN,2024,1,27,2,12,50,2024,1,27,2,15,55,11100,183.56
245744,a0ee9f7974186536fc197f59353a6442,XA-VAE,10,40,4,1,2024-03-21 06:20:00,2024-03-21 10:10:00,186,154.79,NaN,2024,3,21,4,6,20,2024,3,21,4,10,10,13800,154.79


## Venta de productos:

In [122]:
dataSales

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,7396e43fb94caca87add7bb1f1611e7e,Vivabus,Vivabus_Gdl.-Nvo. C. Camionera,4,320.0
...,...,...,...,...,...
2564356,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2564357,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2564358,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2564359,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


### Limpieza:
Se eliminan todo lo que no sea un producto que se consuma en el avión.

In [123]:
dataSales['ProductType'].unique()

array(['Botanas', 'Licores', 'Vivabus', 'Transportaciones CUN',
       'Galletas', 'Specials', 'Bebidas Calientes', 'COMBOS CREW',
       'Hertz.', 'OFERTAS ', 'Transportaciones MTY',
       'Transportaciones TLC', 'VIVA PLAY', 'VIVA Taxis', 'Antros',
       'VivaTransfer', 'Alimentos Charter', 'Perecederos', 'Refrescos',
       'Sopas', 'Lacteos'], dtype=object)

In [124]:
columnsVars = ['Botanas', 'Refrescos', 'Licores', 'Sopas', 'Galletas', 'Bebidas Calientes', 'Perecederos', 'Lacteos']
dataSales = dataSales[dataSales['ProductType'].isin(columnsVars)]

In [125]:
dataSales = dataSales[['Flight_ID','ProductType','ProductName','Quantity']]

Se unifican las bases de datos de los vuelos con los productos comprados en los vuelos.

In [126]:
dataFlightSales2023 = pd.merge(dataSales, dataFlights2023_encoded, on='Flight_ID', how='inner')

In [127]:
dataFlightSales2023

,Flight_ID,ProductType,ProductName,Quantity,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
1,a05290288259526edd3601160b10e1de,Botanas,Cheetos,2,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
2,a05290288259526edd3601160b10e1de,Botanas,Ruffles Queso,6,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
3,a05290288259526edd3601160b10e1de,Refrescos,Coca Sin Azucar,2,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
4,a05290288259526edd3601160b10e1de,Licores,Jack And Coke,1,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030401,6113fc0448cd548cd7e7d3f76e7872c5,Perecederos,Cuerno Clasico De Pavo,5,23,4,18,1,0,2023-11-02 23:10:00,2023-11-03 00:50:00,240,154.0,119.0,2023,11,2,4,23,10,2023,11,3,0,0,50,6000
2030402,78b84bc241b924d6c89e171a34b0f60c,Perecederos,Cuerno Clasico De Pavo,1,64,15,37,3,1,2023-10-05 06:00:00,2023-10-05 07:05:00,240,188.0,150.0,2023,10,5,4,6,0,2023,10,5,4,7,5,3900
2030403,2d7d30a16334ee2db84e5bc027a4a186,Licores,Vino Tinto Sangre De Toro,1,5,10,25,4,1,2023-05-16 18:45:00,2023-05-16 19:35:00,180,149.0,112.0,2023,5,16,5,18,45,2023,5,16,5,19,35,3000
2030404,7a56b895719a91ca6a98004cff956b7d,Licores,Vino Tinto Sangre De Toro,1,34,8,18,1,3,2023-02-15 07:00:00,2023-02-15 08:25:00,220,210.0,170.0,2023,2,15,6,7,0,2023,2,15,6,8,25,5100


In [128]:
dataFlightSales2023_encoded = dataFlightSales2023.copy()

# Se genera el label encoder en cada columna.
for col in ['ProductType', 'ProductName']:
    label_encoders[col] = LabelEncoder()
    dataFlightSales2023_encoded[col] = label_encoders[col].fit_transform(dataFlightSales2023_encoded[col])

dataFlightSales2023_encoded # Datos modificados

,Flight_ID,ProductType,ProductName,Quantity,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration
0,a05290288259526edd3601160b10e1de,1,12,1,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
1,a05290288259526edd3601160b10e1de,1,15,2,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
2,a05290288259526edd3601160b10e1de,1,74,6,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
3,a05290288259526edd3601160b10e1de,6,21,2,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
4,a05290288259526edd3601160b10e1de,4,42,1,55,18,10,1,1,2023-08-28 19:20:00,2023-08-28 20:45:00,186,131.0,100.0,2023,8,28,1,19,20,2023,8,28,1,20,45,5100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030401,6113fc0448cd548cd7e7d3f76e7872c5,5,25,5,23,4,18,1,0,2023-11-02 23:10:00,2023-11-03 00:50:00,240,154.0,119.0,2023,11,2,4,23,10,2023,11,3,0,0,50,6000
2030402,78b84bc241b924d6c89e171a34b0f60c,5,25,1,64,15,37,3,1,2023-10-05 06:00:00,2023-10-05 07:05:00,240,188.0,150.0,2023,10,5,4,6,0,2023,10,5,4,7,5,3900
2030403,2d7d30a16334ee2db84e5bc027a4a186,4,95,1,5,10,25,4,1,2023-05-16 18:45:00,2023-05-16 19:35:00,180,149.0,112.0,2023,5,16,5,18,45,2023,5,16,5,19,35,3000
2030404,7a56b895719a91ca6a98004cff956b7d,4,95,1,34,8,18,1,3,2023-02-15 07:00:00,2023-02-15 08:25:00,220,210.0,170.0,2023,2,15,6,7,0,2023,2,15,6,8,25,5100


### Modelos para predicción de venta de productos:

Se dividen las variables a utilizar como X (Independientes) y variable a obtener Y (Dependiente).

X:
- 'ProductType'
- 'ProductName'
- 'DepartureStation'
- 'ArrivalStation'
- 'Destination_Type'
- 'Origin_Type'
- 'Capacity'
- 'STD_month'
- 'STD_day'
- 'STD_day_week'
- 'STD_hour'
- 'Flight_Duration'
- 'Passengers'

Y:
- 'Quantity'

In [133]:
X = dataFlightSales2023_encoded[['ProductType', 'ProductName', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'Capacity', 'STD_month', 'STD_day', 'STD_day_week', 'STD_hour', 'Flight_Duration', 'Passengers']].head(150000)
y = dataFlightSales2023_encoded['Quantity'].head(150000)

Se generan los grupos para el entrenamiento y comprobación de los modelos y se estandarizan:

In [134]:
from sklearn.model_selection import train_test_split # Division de grupos para entrenamiento y pruebas.
from sklearn.preprocessing import StandardScaler # Estandarización
from sklearn.metrics import mean_squared_error # Corroboración del modelo.

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
# Estandarización de datos.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Random Forest Regressor

In [137]:
from sklearn.ensemble import RandomForestRegressor

In [155]:
# Se elige el modelo y se entrena.
modelSales = RandomForestRegressor()
modelSales.fit(X_train_scaled, y_train)

# Se generan las predicciones.
predictions = modelSales.predict(X_test_scaled)

# Se calculan los errores:
mse = mean_squared_error(y_test, predictions)
print("Error medio cuadrado:", mse)
rmse = np.sqrt(mse)
print("Error medio:", rmse)

# R^2
r2 = r2_score(y_test, predictions)

# R^2
print("R^2:", r2)

Error medio cuadrado: 4.40771708
Error medio: 2.099456377255789
R^2: 0.5226581528985974


### BaggingRegressor

In [139]:
from sklearn.ensemble import BaggingRegressor

In [140]:
# Se elige el modelo y se entrena.
model = BaggingRegressor()
model.fit(X_train_scaled, y_train)

# Se generan las predicciones.
predictions = model.predict(X_test_scaled)

# Se calculan los errores:
mse = mean_squared_error(y_test, predictions)
print("Error medio cuadrado:", mse)
rmse = np.sqrt(mse)
print("Error medio:", rmse)

# R^2
r2 = r2_score(y_test, predictions)

# R^2
print("R^2:", r2)

Error medio cuadrado: 4.808658
Error medio: 2.192865248938019
R^2: 0.47923751680565296


In [141]:
dataFlightSales2023_encoded.describe()

,ProductType,ProductName,Quantity,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration
count,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2030406,2030406,2.030406e+06,2.030406e+06,2.030406e+06,2030406.0,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06,2.030406e+06
mean,3.541575e+00,4.514401e+01,2.464450e+00,4.274947e+01,1.766296e+01,1.726364e+01,2.011922e+00,1.848697e+00,2023-07-08 16:11:20.546097920,2023-07-08 18:27:48.517173504,2.102480e+02,1.887561e+02,1.133334e+02,2023.0,6.725073e+00,1.578247e+01,2.915927e+00,1.374761e+01,2.599020e+01,2.023000e+03,6.725697e+00,1.578363e+01,2.912606e+00,1.473102e+01,2.774893e+01,8.187971e+03
min,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2023-01-02 00:20:00,2023-01-02 02:45:00,1.780000e+02,3.000000e+00,3.000000e+00,2023.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.023000e+03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+02
25%,1.000000e+00,2.000000e+01,1.000000e+00,2.500000e+01,1.000000e+01,1.000000e+01,1.000000e+00,1.000000e+00,2023-04-13 17:17:30,2023-04-13 19:35:00,1.860000e+02,1.680000e+02,9.200000e+01,2023.0,4.000000e+00,8.000000e+00,1.000000e+00,1.000000e+01,1.000000e+01,2.023000e+03,4.000000e+00,8.000000e+00,1.000000e+00,1.200000e+01,1.000000e+01,5.400000e+03
50%,4.000000e+00,4.400000e+01,2.000000e+00,4.000000e+01,1.500000e+01,1.500000e+01,1.000000e+00,1.000000e+00,2023-07-12 10:50:00,2023-07-12 13:20:00,2.200000e+02,1.870000e+02,1.110000e+02,2023.0,7.000000e+00,1.600000e+01,3.000000e+00,1.400000e+01,2.500000e+01,2.023000e+03,7.000000e+00,1.600000e+01,3.000000e+00,1.500000e+01,2.500000e+01,7.200000e+03
75%,6.000000e+00,7.400000e+01,3.000000e+00,6.400000e+01,2.400000e+01,2.400000e+01,4.000000e+00,3.000000e+00,2023-10-03 06:20:00,2023-10-03 08:15:00,2.400000e+02,2.170000e+02,1.330000e+02,2023.0,1.000000e+01,2.300000e+01,5.000000e+00,1.800000e+01,4.000000e+01,2.023000e+03,1.000000e+01,2.300000e+01,5.000000e+00,1.900000e+01,4.500000e+01,1.110000e+04
max,7.000000e+00,9.700000e+01,8.500000e+01,8.300000e+01,4.000000e+01,4.000000e+01,4.000000e+00,4.000000e+00,2023-12-31 23:50:00,2024-01-01 05:15:00,2.400000e+02,2.650000e+02,2.180000e+02,2023.0,1.200000e+01,3.100000e+01,6.000000e+00,2.300000e+01,5.900000e+01,2.024000e+03,1.200000e+01,3.100000e+01,6.000000e+00,2.300000e+01,5.900000e+01,9.840000e+04
std,2.311103e+00,2.862069e+01,2.643081e+00,2.243067e+01,9.794204e+00,9.874832e+00,1.461823e+00,1.400776e+00,NaN,NaN,2.557991e+01,3.544513e+01,2.927339e+01,0.0,3.342607e+00,8.809647e+00,2.024272e+00,4.946589e+00,1.761844e+01,9.415116e-03,3.342407e+00,8.808940e+00,2.023970e+00,5.550766e+00,1.758734e+01,3.948324e+03


## Predicciones:

### Carga de dataset a predecir:

In [165]:
dataFlightsSales2024_encoded = dataFlights2024_encoded.head(5)
# Agregar columnas datos productos

dataFlightsSales2024_encoded['ProductType'] = ['Licores', 'Galletas', 'Botanas', 'Botanas', 'Licores' ]
dataFlightsSales2024_encoded['ProductName'] = ['Baileys', 'Quaker Granola', 'Go Nuts', 'Frutos Secos Enchilados', 'Jack And Coke']

/var/folders/2t/33pmpjk909357lg39f0kql200000gn/T/ipykernel_42757/1013207915.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFlightsSales2024_encoded['ProductType'] = ['Licores', 'Galletas', 'Botanas', 'Botanas', 'Licores' ]
/var/folders/2t/33pmpjk909357lg39f0kql200000gn/T/ipykernel_42757/1013207915.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFlightsSales2024_encoded['ProductName'] = ['Baileys', 'Quaker Granola', 'Go Nuts', 'Frutos Secos Enchilados', 'Jack And Coke']


In [164]:
dataFlightsSales2024_encoded

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration,Expected Passengers,ProductType,ProductName
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,18,14,3,1,2024-02-16 17:10:00,2024-02-16 17:55:00,220,193.68,NaN,2024,2,16,0,17,10,2024,2,16,0,17,55,2700,193.68,Licores,Baileys
19,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,22,0,4,1,2024-01-12 08:20:00,2024-01-12 09:20:00,186,178.40,NaN,2024,1,12,0,8,20,2024,1,12,0,9,20,3600,178.40,Galletas,Quaker Granola
25,85d3b182431d323b856e50e249689981,XA-VIY,15,5,4,1,2024-03-18 10:30:00,2024-03-18 13:25:00,186,168.98,NaN,2024,3,18,1,10,30,2024,3,18,1,13,25,10500,168.98,Botanas,Go Nuts
26,04e83b220f5513bd6cc636784dcddec6,XA-VXC,34,17,3,0,2024-04-05 00:50:00,2024-04-05 04:45:00,240,196.28,NaN,2024,4,5,0,0,50,2024,4,5,0,4,45,14100,196.28,Botanas,Frutos Secos Enchilados
28,0dbc47cfbd0af71db43ef3aa60ca1ced,XA-VII,22,21,0,1,2024-02-24 07:35:00,2024-02-24 09:20:00,186,182.82,NaN,2024,2,24,2,7,35,2024,2,24,2,9,20,6300,182.82,Licores,Jack And Coke


In [151]:
for col in ['ProductType', 'ProductName']:
    dataFlightsSales2024_encoded[col] = label_encoders[col].transform(dataFlightsSales2024_encoded[col])

/var/folders/2t/33pmpjk909357lg39f0kql200000gn/T/ipykernel_42757/622502190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFlightsSales2024_encoded[col] = label_encoders[col].transform(dataFlightsSales2024_encoded[col])
/var/folders/2t/33pmpjk909357lg39f0kql200000gn/T/ipykernel_42757/622502190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFlightsSales2024_encoded[col] = label_encoders[col].transform(dataFlightsSales2024_encoded[col])


In [152]:
X_pred_sales = dataFlightsSales2024_encoded[['ProductType', 'ProductName', 'DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type', 'Capacity', 'STD_month', 'STD_day', 'STD_day_week', 'STD_hour', 'Flight_Duration', 'Passengers']]

In [153]:
# Estandarización de datos.
scaler = StandardScaler()
X_finalpred_sales = scaler.fit_transform(X_pred_sales)

In [156]:
# Se generan las predicciones.
y_predictions_sales = modelSales.predict(X_finalpred_sales)

In [159]:
dataFlightsSales2024_encoded['Expected Quantity Sales'] = y_predictions_sales

/var/folders/2t/33pmpjk909357lg39f0kql200000gn/T/ipykernel_42757/811721809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFlightsSales2024_encoded['Expected Quantity Sales'] = y_predictions_sales


In [160]:
dataFlightsSales2024_encoded

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_year,STD_month,STD_day,STD_day_week,STD_hour,STD_minute,STA_year,STA_month,STA_day,STA_day_week,STA_hour,STA_minute,Flight_Duration,ProductType,ProductName,Quantity,Expected Quantity Sales
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,18,14,3,1,2024-02-16 17:10:00,2024-02-16 17:55:00,220,193.68,NaN,2024,2,16,0,17,10,2024,2,16,0,17,55,2700,4,5,2.59,2.59
19,bb319ee8f62b5ae38e6f6da4dbff03cb,XA-VAV,22,0,4,1,2024-01-12 08:20:00,2024-01-12 09:20:00,186,178.40,NaN,2024,1,12,0,8,20,2024,1,12,0,9,20,3600,2,70,1.17,1.17
25,85d3b182431d323b856e50e249689981,XA-VIY,15,5,4,1,2024-03-18 10:30:00,2024-03-18 13:25:00,186,168.98,NaN,2024,3,18,1,10,30,2024,3,18,1,13,25,10500,1,37,1.20,1.20
26,04e83b220f5513bd6cc636784dcddec6,XA-VXC,34,17,3,0,2024-04-05 00:50:00,2024-04-05 04:45:00,240,196.28,NaN,2024,4,5,0,0,50,2024,4,5,0,4,45,14100,1,33,1.84,1.84
28,0dbc47cfbd0af71db43ef3aa60ca1ced,XA-VII,22,21,0,1,2024-02-24 07:35:00,2024-02-24 09:20:00,186,182.82,NaN,2024,2,24,2,7,35,2024,2,24,2,9,20,6300,4,42,2.43,2.43
